In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from pylab import rcParams

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

# keras imports - comment them out or do `pip install keras`
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

# gensim for pretrained word2vec model
import gensim

# for synopsis clean up
import string

# list of stopwords used by MySQL in MyISAM
stop_words = ["a's" , "able" , "about" , "above" , "according" , "accordingly" , "across" , "actually" , "after" , "afterwards" , "again" , "against" , "ain't" , "all" , "allow" , "allows" , "almost" , "alone" , "along" , "already" , "also" , "although" , "always" , "am" , "among" , "amongst" , "an" , "and" , "another" , "any" , "anybody" , "anyhow" , "anyone" , "anything" , "anyway" , "anyways" , "anywhere" , "apart" , "appear" , "appreciate" , "appropriate" , "are" , "aren't" , "around" , "as" , "aside" , "ask" , "asking" , "associated" , "at" , "available" , "away" , "awfully" , "be" , "became" , "because" , "become" , "becomes" , "becoming" , "been" , "before" , "beforehand" , "behind" , "being" , "believe" , "below" , "beside" , "besides" , "best" , "better" , "between" , "beyond" , "both" , "brief" , "but" , "by" , "c'mon" , "c's" , "came" , "can" , "can't" , "cannot" , "cant" , "cause" , "causes" , "certain" , "certainly" , "changes" , "clearly" , "co" , "com" , "come" , "comes" , "concerning" , "consequently" , "consider" , "considering" , "contain" , "containing" , "contains" , "corresponding" , "could" , "couldn't" , "course" , "currently" , "definitely" , "described" , "despite" , "did" , "didn't" , "different" , "do" , "does" , "doesn't" , "doing" , "don't" , "done" , "down" , "downwards" , "during" , "each" , "edu" , "eg" , "eight" , "either" , "else" , "elsewhere" , "enough" , "entirely" , "especially" , "et" , "etc" , "even" , "ever" , "every" , "everybody" , "everyone" , "everything" , "everywhere" , "ex" , "exactly" , "example" , "except" , "far" , "few" , "fifth" , "first" , "five" , "followed" , "following" , "follows" , "for" , "former" , "formerly" , "forth" , "four" , "from" , "further" , "furthermore" , "get" , "gets" , "getting" , "given" , "gives" , "go" , "goes" , "going" , "gone" , "got" , "gotten" , "greetings" , "had" , "hadn't" , "happens" , "hardly" , "has" , "hasn't" , "have" , "haven't" , "having" , "he" , "he's" , "hello" , "help" , "hence" , "her" , "here" , "here's" , "hereafter" , "hereby" , "herein" , "hereupon" , "hers" , "herself" , "hi" , "him" , "himself" , "his" , "hither" , "hopefully" , "how" , "howbeit" , "however" , "i'd" , "i'll" , "i'm" , "i've" , "ie" , "if" , "ignored" , "immediate" , "in" , "inasmuch" , "inc" , "indeed" , "indicate" , "indicated" , "indicates" , "inner" , "insofar" , "instead" , "into" , "inward" , "is" , "isn't" , "it" , "it'd" , "it'll" , "it's" , "its" , "itself" , "just" , "keep" , "keeps" , "kept" , "know" , "known" , "knows" , "last" , "lately" , "later" , "latter" , "latterly" , "least" , "less" , "lest" , "let" , "let's" , "like" , "liked" , "likely" , "little" , "look" , "looking" , "looks" , "ltd" , "mainly" , "many" , "may" , "maybe" , "me" , "mean" , "meanwhile" , "merely" , "might" , "more" , "moreover" , "most" , "mostly" , "much" , "must" , "my" , "myself" , "name" , "namely" , "nd" , "near" , "nearly" , "necessary" , "need" , "needs" , "neither" , "never" , "nevertheless" , "new" , "next" , "nine" , "no" , "nobody" , "non" , "none" , "noone" , "nor" , "normally" , "not" , "nothing" , "novel" , "now" , "nowhere" , "obviously" , "of" , "off" , "often" , "oh" , "ok" , "okay" , "old" , "on" , "once" , "one" , "ones" , "only" , "onto" , "or" , "other" , "others" , "otherwise" , "ought" , "our" , "ours" , "ourselves" , "out" , "outside" , "over" , "overall" , "own" , "particular" , "particularly" , "per" , "perhaps" , "placed" , "please" , "plus" , "possible" , "presumably" , "probably" , "provides" , "que" , "quite" , "qv" , "rather" , "rd" , "re" , "really" , "reasonably" , "regarding" , "regardless" , "regards" , "relatively" , "respectively" , "right" , "said" , "same" , "saw" , "say" , "saying" , "says" , "second" , "secondly" , "see" , "seeing" , "seem" , "seemed" , "seeming" , "seems" , "seen" , "self" , "selves" , "sensible" , "sent" , "serious" , "seriously" , "seven" , "several" , "shall" , "she" , "should" , "shouldn't" , "since" , "six" , "so" , "some" , "somebody" , "somehow" , "someone" , "something" , "sometime" , "sometimes" , "somewhat" , "somewhere" , "soon" , "sorry" , "specified" , "specify" , "specifying" , "still" , "sub" , "such" , "sup" , "sure" , "t's" , "take" , "taken" , "tell" , "tends" , "th" , "than" , "thank" , "thanks" , "thanx" , "that" , "that's" , "thats" , "the" , "their" , "theirs" , "them" , "themselves" , "then" , "thence" , "there" , "there's" , "thereafter" , "thereby" , "therefore" , "therein" , "theres" , "thereupon" , "these" , "they" , "they'd" , "they'll" , "they're" , "they've" , "think" , "third" , "this" , "thorough" , "thoroughly" , "those" , "though" , "three" , "through" , "throughout" , "thru" , "thus" , "to" , "together" , "too" , "took" , "toward" , "towards" , "tried" , "tries" , "truly" , "try" , "trying" , "twice" , "two" , "un" , "under" , "unfortunately" , "unless" , "unlikely" , "until" , "unto" , "up" , "upon" , "us" , "use" , "used" , "useful" , "uses" , "using" , "usually" , "value" , "various" , "very" , "via" , "viz" , "vs" , "want" , "wants" , "was" , "wasn't" , "way" , "we" , "we'd" , "we'll" , "we're" , "we've" , "welcome" , "well" , "went" , "were" , "weren't" , "what" , "what's" , "whatever" , "when" , "whence" , "whenever" , "where" , "where's" , "whereafter" , "whereas" , "whereby" , "wherein" , "whereupon" , "wherever" , "whether" , "which" , "while" , "whither" , "who" , "who's" , "whoever" , "whole" , "whom" , "whose" , "why" , "will" , "willing" , "wish" , "with" , "within" , "without" , "won't" , "wonder" , "would" , "wouldn't" , "yes" , "yet" , "you" , "you'd" , "you'll" , "you're" , "you've" , "your" , "yours" , "yourself" , "yourselves" , "zero"]

rcParams['figure.figsize'] = 10, 6
display(HTML("<style>.container { width:95% !important; }</style>"))

Using TensorFlow backend.
unable to import 'smart_open.gcs', disabling that module


In [2]:
df = pd.read_csv("data/tidy_anime.csv")
df.shape

(77911, 28)

In [3]:
desired_cols = ['animeID', 'title_english', 'type', 'source', 'producers', 'genre', 'studio',
               'episodes', 'premiered', 'rating', 'score', 'scored_by', 'rank', 'popularity',
               'members', 'favorites', 'synopsis']
truncated_df = df[desired_cols]

In [4]:
desired_cols = ['animeID', 'title_english', 'type', 'source', 'producers', 'genre', 'studio',
               'episodes', 'premiered', 'rating', 'score', 'scored_by', 'rank', 'popularity',
               'members', 'favorites', 'synopsis']
truncated_df = df[desired_cols]
truncated_df.head()

,animeID,title_english,type,source,producers,genre,studio,episodes,premiered,rating,score,scored_by,rank,popularity,members,favorites,synopsis
0,1,Cowboy Bebop,TV,Original,Bandai Visual,Action,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
1,1,Cowboy Bebop,TV,Original,Bandai Visual,Adventure,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
2,1,Cowboy Bebop,TV,Original,Bandai Visual,Comedy,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
3,1,Cowboy Bebop,TV,Original,Bandai Visual,Drama,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
4,1,Cowboy Bebop,TV,Original,Bandai Visual,Sci-Fi,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."


In [5]:
#change NaN to 0 in Premiered column

truncated_df.loc[:,'premiered'] = truncated_df.loc[:,'premiered'].fillna(0)

C:\Users\Rando\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
# filter out bad titles. Only want titles that have an english name

orig_len = len(truncated_df)
filtered_df = truncated_df[truncated_df['title_english'].notnull()]
new_len = len(filtered_df)
print ("removed {} bad anime after filtering for english titled anime only".format(orig_len - new_len))

# drop NaN rows
filtered_df.dropna(inplace=True)
print ("removed {} bad anime after dropping NaN rows".format(new_len - len(filtered_df)))


removed 30430 bad anime after filtering for english titled anime only
removed 8678 bad anime after dropping NaN rows


C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [7]:
# currently the anime is duplicated, one row per genre per studio. We need to flatten all to one row
# also need to do this for type, source, producers, rating

all_ids = set(filtered_df['animeID'].unique()) # 1.8K anime IDs
print ("{} unique anime".format(len(all_ids)))

id_genre_mapping = {}
for each_id in all_ids:
    genre_list = list(filtered_df[truncated_df['animeID'] == each_id]['genre'])
    id_genre_mapping[each_id] = genre_list
    
id_studio_mapping = {}
for each_id in all_ids:
    id_studio_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['studio'])
    
id_source_mapping = {}
for each_id in all_ids:
    id_source_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['source'])
    
id_producers_mapping = {}
for each_id in all_ids:
    id_producers_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['producers'])
    
id_rating_mapping = {}
for each_id in all_ids:
    id_rating_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['rating'])
    
id_type_mapping = {}
for each_id in all_ids:
    id_type_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['type'])

2855 unique anime


C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [8]:
# get distinct df, remove duplicates
reduced_df = filtered_df.groupby('animeID').head(1)

In [9]:
# will add 40 columns to the data
all_genres = sorted(list(set([item for sublist in id_genre_mapping.values() for item in sublist])))
all_studios = sorted(list(set([item for sublist in id_studio_mapping.values() for item in sublist])))
all_sources = sorted(list(set([item for sublist in id_source_mapping.values() for item in sublist])))
all_producers = sorted(list(set([item for sublist in id_producers_mapping.values() for item in sublist])))
all_ratings = sorted(list(set([item for sublist in id_rating_mapping.values() for item in sublist])))
all_types = sorted(list(set([item for sublist in id_type_mapping.values() for item in sublist])))



In [10]:
anime_IDs = reduced_df.animeID.tolist()
genres_new = []
studios_new = []
sources_new = []
producers_new = []
ratings_new = []
types_new = []
for each_id in anime_IDs:
    genres_new.append(id_genre_mapping[each_id])
    studios_new.append(id_studio_mapping[each_id])
    sources_new.append(id_source_mapping[each_id])
    producers_new.append(id_producers_mapping[each_id])
    ratings_new.append(id_rating_mapping[each_id])
    types_new.append(id_type_mapping[each_id])
reduced_df.head()

,animeID,title_english,type,source,producers,genre,studio,episodes,premiered,rating,score,scored_by,rank,popularity,members,favorites,synopsis
0,1,Cowboy Bebop,TV,Original,Bandai Visual,Action,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
6,5,Cowboy Bebop: The Movie,Movie,Original,Sunrise,Action,Bones,1.0,0,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776,"Another day, another bounty—such is the life o..."
16,6,Trigun,TV,Manga,Victor Entertainment,Action,Madhouse,26.0,Spring 1998,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432,"Vash the Stampede is the man with a $$60,000,0..."
22,7,Witch Hunter Robin,TV,Original,Bandai Visual,Action,Sunrise,26.0,Summer 2002,PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537,Witches are individuals with special powers li...
28,8,Beet the Vandel Buster,TV,Manga,TV Tokyo,Adventure,Toei Animation,52.0,Fall 2004,PG - Children,7.03,4894,3544,3704,11708,14,It is the dark century and the people are suff...


In [11]:
# pull out english titles and relevant data we want to show in output
info_df = reduced_df[['animeID', 'title_english', 'synopsis']]
# write this data to a new csv so we can reference it later
info_df.to_csv(r'data/relevant_output_data.csv')

In [12]:
mlb = MultiLabelBinarizer()
encoded_genres = mlb.fit_transform(genres_new)
encoded_studios = mlb.fit_transform(studios_new)
encoded_sources = mlb.fit_transform(sources_new)
encoded_producers = mlb.fit_transform(producers_new)
encoded_ratings = mlb.fit_transform(ratings_new)
encoded_types = mlb.fit_transform(types_new)

In [13]:
genre_columns_added = encoded_genres.shape[1]

# for col_idx in range(genre_columns_added):
#     reduced_df.insert(len(reduced_df.columns), "genre_{}".format(col_idx+1), encoded_genres[:, col_idx])
    
for col_idx in range(genre_columns_added):
    reduced_df.insert(len(reduced_df.columns), "genre_{}".format(all_genres[col_idx]), encoded_genres[:, col_idx])

for col_idx in range(encoded_studios.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "studio_{}".format(all_studios[col_idx]), encoded_studios[:, col_idx])
    
for col_idx in range(encoded_sources.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "source_{}".format(all_sources[col_idx]), encoded_sources[:, col_idx])
    
for col_idx in range(encoded_producers.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "producer_{}".format(all_producers[col_idx]), encoded_producers[:, col_idx])
    
for col_idx in range(encoded_ratings.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "rating_{}".format(all_ratings[col_idx]), encoded_ratings[:, col_idx])
    
for col_idx in range(encoded_types.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "type_{}".format(all_types[col_idx]), encoded_types[:, col_idx])

In [14]:
#### Decreasing Count of Entries for each column value

#set options to see full dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


ranked_genres = reduced_df.groupby('genre').count().sort_values(by=["animeID"], ascending=False)['animeID'].keys().tolist()
#genres_to_drop = ranked_genres[21:]

for genre in ranked_genres[:21]:
    all_genres.remove(genre)
    #Keep top 15? Add Horror, Historical?
genres_to_drop = ["genre_" + genre for genre in all_genres]

ranked_studios = reduced_df.groupby('studio').count().sort_values(by=["animeID"], ascending=False)['animeID'].keys().tolist()
#studios_to_drop = ranked_studios[36:]
    #Keep studios with 20+ records?

for studio in ranked_studios[:36]:
    all_studios.remove(studio)
    #Keep top 15? Add Horror, Historical?
studios_to_drop = ["studio_" + studio for studio in all_studios]

ranked_sources = reduced_df.groupby('source').count().sort_values(by=["animeID"], ascending=False)['animeID'].keys().tolist()
#sources_to_drop = ranked_sources[7:]
#     #Keep all 15?

for source in ranked_sources[:7]:
    all_sources.remove(source)
sources_to_drop = ["source_" + source for source in all_sources]


ranked_producers = reduced_df.groupby('producers').count().sort_values(by=["animeID"], ascending=False)['animeID'].keys().tolist()
#producers_to_drop = ranked_producers[25:]
#     #Keep Producers with 30+ records?
for producer in ranked_producers[:25]:
    all_producers.remove(producer)
producers_to_drop = ["producer_" + producer for producer in all_producers]

ranked_ratings = reduced_df.groupby('rating').count().sort_values(by=["animeID"], ascending=False)['animeID'].keys().tolist()
    #Keep all except 'None'?
    
for rating in ranked_ratings[:5]:
    all_ratings.remove(rating)
ratings_to_drop = ["rating_" + rating for rating in all_ratings]

    
#reduced_df.groupby('type').count().sort_values(by=["animeID"], ascending=False)['animeID']

columns_to_drop = genres_to_drop + studios_to_drop + sources_to_drop + producers_to_drop + ratings_to_drop

len(columns_to_drop)


C:\Users\Rando\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


991

In [15]:
reduced_df = reduced_df.drop(['type', 'source', 'producers', 'genre', 'studio', 'rating'], axis = 1)
reduced_df.to_csv(r'data/one_hot_encode_complete.csv')

reduced_df = reduced_df.drop(columns_to_drop, axis = 1)
reduced_df.to_csv(r'data/one_hot_encode_reduced.csv')

reduced_df.head()

,animeID,title_english,episodes,premiered,score,scored_by,rank,popularity,members,favorites,synopsis,genre_Action,genre_Adventure,genre_Comedy,genre_Dementia,genre_Drama,genre_Ecchi,genre_Fantasy,genre_Game,genre_Harem,genre_Historical,genre_Horror,genre_Kids,genre_Magic,genre_Military,genre_Music,genre_Mystery,genre_Psychological,genre_Romance,genre_Sci-Fi,genre_Slice of Life,genre_Sports,"studio_""Brains Base""",studio_A-1 Pictures,studio_AIC,studio_Arms,studio_Bones,studio_Diomedea,studio_Doga Kobo,studio_Gainax,studio_Gonzo,studio_J.C.Staff,studio_Kyoto Animation,studio_MAPPA,studio_Madhouse,studio_Manglobe,studio_Nippon Animation,studio_OLM,studio_P.A. Works,studio_Production I.G,studio_Production Reed,studio_Satelight,studio_Shaft,studio_Shin-Ei Animation,studio_Silver Link.,studio_Studio Deen,studio_Studio Gallop,studio_Studio Gokumi,studio_Studio Pierrot,studio_Sunrise,studio_TMS Entertainment,studio_TNK,studio_Tatsunoko Production,studio_Toei Animation,studio_Xebec,studio_Zexcs,studio_feel.,studio_ufotable,source_Game,source_Light novel,source_Manga,source_Novel,source_Original,source_Unknown,source_Visual novel,producer_Aniplex,producer_Bandai Visual,producer_DAX Production,producer_Dentsu,producer_Frontier Works,producer_Fuji TV,producer_Genco,producer_Geneon Universal Entertainment,producer_Kadokawa Shoten,producer_Lantis,producer_Media Factory,producer_Movic,producer_NHK,producer_Nihon Ad Systems,producer_Pony Canyon,producer_Production I.G,producer_Shochiku,producer_Shogakukan Productions,producer_Sotsu,producer_Starchild Records,producer_TBS,producer_TV Asahi,producer_TV Tokyo,producer_VAP,producer_Victor Entertainment,rating_G - All Ages,rating_PG - Children,rating_PG-13 - Teens 13 or older,rating_R - 17+ (violence & profanity),rating_R+ - Mild Nudity,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,1,Cowboy Bebop,26.0,Spring 1998,8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as ""Cowboys."" The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member's dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Bebop is a space Western classic and an homage to the smooth and improvised music it is named after. [Written by MAL Rewrite]",1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
6,5,Cowboy Bebop: The Movie,1.0,0,8.41,120243,164,449,197791,776,"Another day, another bounty—such is the life of the often unlucky crew of the Bebop. However, this routine is interrupted when Faye, who is chasing a fairly worthless target on Mars, witnesses an oil tanker suddenly explode, causing mass hysteria. As casualties mount due to a strange disease spreading through the smoke from the blast, a whopping three hundred million woolong price is placed on the head of the supposed perpetrator. With lives at stake and a solution to their money problems in sight, the Bebop crew springs into action. Spike, Jet, Faye, and Edward, followed closely by Ein, split up to pursue different leads across Alba City. Through their individual investigations, they discover a cover-up sch

# Encoding textual data - tokenization approach
* May want to look into cleaning all the synopsis first, ex: remove (), lower case, etc (disregarding this for now)

In [16]:
synopsis_list = reduced_df['synopsis'].tolist()
# synopsis_list

In [17]:
VOCAB_SIZE = None
MAX_SEQ_LEN = 0

# find vocab_size
all_words = {}

# DEBUG
debug = False
init = 0
count = 100

for synopsis in synopsis_list:

    # =========================
    if debug:
        if init == count - 1:
            print("Original Synopsis:")
            print(synopsis + "\n")
    # =========================
    
    # deletes synopsis credits e.g. [Written by MAL Rewrite], (Source: ANN)
    if synopsis[-1] == ")":
        idx = synopsis.rfind("(")
        synopsis = synopsis[:idx]
    elif synopsis[-1] == "]":
        idx = synopsis.rfind("[")
        synopsis = synopsis[:idx]
    
    # deletes punctuation
    punctuation = string.punctuation
    for c in punctuation:
        synopsis = synopsis.replace(c, "")
    
    synopsis = synopsis.lower()
    
    # =========================
    if debug:
        if init == count - 1:
            print("Lower Case Synopsis Without Punctuation:")
            print(synopsis + "\n")
    # =========================

    word_list = synopsis.split(" ")
    
    
    # =========================
    if debug:
        if init == count - 1:
            print("Original Word List:")
            print(str(word_list) + "\n")
    # =========================
    
    # remove stop words
    word_list = [x for x in word_list if x not in stop_words]
    
    
    # =========================
    if debug:
        if init == count - 1:
            print("Word List Without Stop Words:")
            print(str(word_list) + "\n")
        init += 1
    # =========================
    
    # find max seq len
    if len(word_list) > MAX_SEQ_LEN:
        MAX_SEQ_LEN = len(word_list)
        sent = word_list

    for ea_word in word_list:
        
        if ea_word in all_words:
            all_words[ea_word] += 1
        else:
            all_words[ea_word] = 1
VOCAB_SIZE = len(all_words.keys())
print ('vocab_size = ', VOCAB_SIZE)
print ('max_seq_len = ', MAX_SEQ_LEN)
# print (MAX_SEQ_LEN, sent)

vocab_size =  25753
max_seq_len =  290


In [18]:
encoded_synopsis = [one_hot(x, VOCAB_SIZE) for x in synopsis_list]
padded_synopsis = pad_sequences(encoded_synopsis, maxlen=MAX_SEQ_LEN, padding='pre')
padded_synopsis.shape

(2855, 290)

In [19]:
# load pretrained google word2vec model
pretrained_path = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(pretrained_path, binary=True)
# model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [20]:
# get averaged word embedding
EMBED_DIM = 300

all_synopsis_vectors = np.empty((0, EMBED_DIM))
print (all_synopsis_vectors.shape)
for each_synopsis in synopsis_list:
    synopsis_words = each_synopsis.split(" ")
    word_count = len(synopsis_words)
    synopsis_sum_vector = np.zeros((EMBED_DIM,))
    for each_word in synopsis_words:
        try:
            synopsis_sum_vector += model[each_word]
        except:
            # word not in pretrained vocab
            pass
    synopsis_avg_vector = (synopsis_sum_vector / word_count).reshape(1, -1)
    all_synopsis_vectors = np.append(all_synopsis_vectors, synopsis_avg_vector, axis=0)
all_synopsis_vectors.shape

(0, 300)


(2855, 300)

In [21]:
synopsis_columns_added = all_synopsis_vectors.shape[1]
for col_idx in range(synopsis_columns_added):
    reduced_df.insert(len(reduced_df.columns), "synopsis_embedded_{}".format(col_idx+1), all_synopsis_vectors[:, col_idx])
reduced_df.head()

,animeID,title_english,episodes,premiered,score,scored_by,rank,popularity,members,favorites,synopsis,genre_Action,genre_Adventure,genre_Comedy,genre_Dementia,genre_Drama,genre_Ecchi,genre_Fantasy,genre_Game,genre_Harem,genre_Historical,genre_Horror,genre_Kids,genre_Magic,genre_Military,genre_Music,genre_Mystery,genre_Psychological,genre_Romance,genre_Sci-Fi,genre_Slice of Life,genre_Sports,"studio_""Brains Base""",studio_A-1 Pictures,studio_AIC,studio_Arms,studio_Bones,studio_Diomedea,studio_Doga Kobo,studio_Gainax,studio_Gonzo,studio_J.C.Staff,studio_Kyoto Animation,studio_MAPPA,studio_Madhouse,studio_Manglobe,studio_Nippon Animation,studio_OLM,studio_P.A. Works,studio_Production I.G,studio_Production Reed,studio_Satelight,studio_Shaft,studio_Shin-Ei Animation,studio_Silver Link.,studio_Studio Deen,studio_Studio Gallop,studio_Studio Gokumi,studio_Studio Pierrot,studio_Sunrise,studio_TMS Entertainment,studio_TNK,studio_Tatsunoko Production,studio_Toei Animation,studio_Xebec,studio_Zexcs,studio_feel.,studio_ufotable,source_Game,source_Light novel,source_Manga,source_Novel,source_Original,source_Unknown,source_Visual novel,producer_Aniplex,producer_Bandai Visual,producer_DAX Production,producer_Dentsu,producer_Frontier Works,producer_Fuji TV,producer_Genco,producer_Geneon Universal Entertainment,producer_Kadokawa Shoten,producer_Lantis,producer_Media Factory,producer_Movic,producer_NHK,producer_Nihon Ad Systems,producer_Pony Canyon,producer_Production I.G,producer_Shochiku,producer_Shogakukan Productions,producer_Sotsu,producer_Starchild Records,producer_TBS,producer_TV Asahi,producer_TV Tokyo,producer_VAP,producer_Victor Entertainment,rating_G - All Ages,rating_PG - Children,rating_PG-13 - Teens 13 or older,rating_R - 17+ (violence & profanity),rating_R+ - Mild Nudity,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,synopsis_embedded_1,synopsis_embedded_2,synopsis_embedded_3,synopsis_embedded_4,synopsis_embedded_5,synopsis_embedded_6,synopsis_embedded_7,synopsis_embedded_8,synopsis_embedded_9,synopsis_embedded_10,synopsis_embedded_11,synopsis_embedded_12,synopsis_embedded_13,synopsis_embedded_14,synopsis_embedded_15,synopsis_embedded_16,synopsis_embedded_17,synopsis_embedded_18,synopsis_embedded_19,synopsis_embedded_20,synopsis_embedded_21,synopsis_embedded_22,synopsis_embedded_23,synopsis_embedded_24,synopsis_embedded_25,synopsis_embedded_26,synopsis_embedded_27,synopsis_embedded_28,synopsis_embedded_29,synopsis_embedded_30,synopsis_embedded_31,synopsis_embedded_32,synopsis_embedded_33,synopsis_embedded_34,synopsis_embedded_35,synopsis_embedded_36,synopsis_embedded_37,synopsis_embedded_38,synopsis_embedded_39,synopsis_embedded_40,synopsis_embedded_41,synopsis_embedded_42,synopsis_embedded_43,synopsis_embedded_44,synopsis_embedded_45,synopsis_embedded_46,synopsis_embedded_47,synopsis_embedded_48,synopsis_embedded_49,synopsis_embedded_50,synopsis_embedded_51,synopsis_embedded_52,synopsis_embedded_53,synopsis_embedded_54,synopsis_embedded_55,synopsis_embedded_56,synopsis_embedded_57,synopsis_embedded_58,synopsis_embedded_59,synopsis_embedded_60,synopsis_embedded_61,synopsis_embedded_62,synopsis_embedded_63,synopsis_embedded_64,synopsis_embedded_65,synopsis_embedded_66,synopsis_embedded_67,synopsis_embedded_68,synopsis_embedded_69,synopsis_embedded_70,synopsis_embedded_71,synopsis_embedded_72,synopsis_embedded_73,synopsis_embedded_74,synopsis_embedded_75,synopsis_embedded_76,synopsis_embedded_77,synopsis_embedded_78,synopsis_embedded_79,synopsis_embedded_80,synopsis_embedded_81,synopsis_embedded_82,synopsis_embedded_83,synopsis_embedded_84,synopsis_embedded_85,synopsis_embedded_86,synopsis_embedded_87,synopsis_embedded_88,synopsis_embedded_89,synopsis_embedded_90,synopsis_embedded_91,synopsis_embedded_92,synopsis_embedded_93,synopsis_embedded_94,synopsis_embedded_95,synopsis_embedded_96,synopsis_embedded_97,synopsis_embedded_98,synopsis_embedded_99,synopsis_embedded_100,synopsis_embedded_101,synopsis_embedded_102,synop

In [104]:
simp_cols = [col for col in list(reduced_df.columns) if ((col != 'title_english') & (col != 'synopsis'))]
more_reduced_df = reduced_df[simp_cols]
print(more_reduced_df.shape)
more_reduced_df.head()

(2855, 409)


,animeID,episodes,premiered,score,scored_by,rank,popularity,members,favorites,genre_Action,genre_Adventure,genre_Comedy,genre_Dementia,genre_Drama,genre_Ecchi,genre_Fantasy,genre_Game,genre_Harem,genre_Historical,genre_Horror,genre_Kids,genre_Magic,genre_Military,genre_Music,genre_Mystery,genre_Psychological,genre_Romance,genre_Sci-Fi,genre_Slice of Life,genre_Sports,"studio_""Brains Base""",studio_A-1 Pictures,studio_AIC,studio_Arms,studio_Bones,studio_Diomedea,studio_Doga Kobo,studio_Gainax,studio_Gonzo,studio_J.C.Staff,studio_Kyoto Animation,studio_MAPPA,studio_Madhouse,studio_Manglobe,studio_Nippon Animation,studio_OLM,studio_P.A. Works,studio_Production I.G,studio_Production Reed,studio_Satelight,studio_Shaft,studio_Shin-Ei Animation,studio_Silver Link.,studio_Studio Deen,studio_Studio Gallop,studio_Studio Gokumi,studio_Studio Pierrot,studio_Sunrise,studio_TMS Entertainment,studio_TNK,studio_Tatsunoko Production,studio_Toei Animation,studio_Xebec,studio_Zexcs,studio_feel.,studio_ufotable,source_Game,source_Light novel,source_Manga,source_Novel,source_Original,source_Unknown,source_Visual novel,producer_Aniplex,producer_Bandai Visual,producer_DAX Production,producer_Dentsu,producer_Frontier Works,producer_Fuji TV,producer_Genco,producer_Geneon Universal Entertainment,producer_Kadokawa Shoten,producer_Lantis,producer_Media Factory,producer_Movic,producer_NHK,producer_Nihon Ad Systems,producer_Pony Canyon,producer_Production I.G,producer_Shochiku,producer_Shogakukan Productions,producer_Sotsu,producer_Starchild Records,producer_TBS,producer_TV Asahi,producer_TV Tokyo,producer_VAP,producer_Victor Entertainment,rating_G - All Ages,rating_PG - Children,rating_PG-13 - Teens 13 or older,rating_R - 17+ (violence & profanity),rating_R+ - Mild Nudity,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,synopsis_embedded_1,synopsis_embedded_2,synopsis_embedded_3,synopsis_embedded_4,synopsis_embedded_5,synopsis_embedded_6,synopsis_embedded_7,synopsis_embedded_8,synopsis_embedded_9,synopsis_embedded_10,synopsis_embedded_11,synopsis_embedded_12,synopsis_embedded_13,synopsis_embedded_14,synopsis_embedded_15,synopsis_embedded_16,synopsis_embedded_17,synopsis_embedded_18,synopsis_embedded_19,synopsis_embedded_20,synopsis_embedded_21,synopsis_embedded_22,synopsis_embedded_23,synopsis_embedded_24,synopsis_embedded_25,synopsis_embedded_26,synopsis_embedded_27,synopsis_embedded_28,synopsis_embedded_29,synopsis_embedded_30,synopsis_embedded_31,synopsis_embedded_32,synopsis_embedded_33,synopsis_embedded_34,synopsis_embedded_35,synopsis_embedded_36,synopsis_embedded_37,synopsis_embedded_38,synopsis_embedded_39,synopsis_embedded_40,synopsis_embedded_41,synopsis_embedded_42,synopsis_embedded_43,synopsis_embedded_44,synopsis_embedded_45,synopsis_embedded_46,synopsis_embedded_47,synopsis_embedded_48,synopsis_embedded_49,synopsis_embedded_50,synopsis_embedded_51,synopsis_embedded_52,synopsis_embedded_53,synopsis_embedded_54,synopsis_embedded_55,synopsis_embedded_56,synopsis_embedded_57,synopsis_embedded_58,synopsis_embedded_59,synopsis_embedded_60,synopsis_embedded_61,synopsis_embedded_62,synopsis_embedded_63,synopsis_embedded_64,synopsis_embedded_65,synopsis_embedded_66,synopsis_embedded_67,synopsis_embedded_68,synopsis_embedded_69,synopsis_embedded_70,synopsis_embedded_71,synopsis_embedded_72,synopsis_embedded_73,synopsis_embedded_74,synopsis_embedded_75,synopsis_embedded_76,synopsis_embedded_77,synopsis_embedded_78,synopsis_embedded_79,synopsis_embedded_80,synopsis_embedded_81,synopsis_embedded_82,synopsis_embedded_83,synopsis_embedded_84,synopsis_embedded_85,synopsis_embedded_86,synopsis_embedded_87,synopsis_embedded_88,synopsis_embedded_89,synopsis_embedded_90,synopsis_embedded_91,synopsis_embedded_92,synopsis_embedded_93,synopsis_embedded_94,synopsis_embedded_95,synopsis_embedded_96,synopsis_embedded_97,synopsis_embedded_98,synopsis_embedded_99,synopsis_embedded_100,synopsis_embedded_101,synopsis_embedded_102,synopsis_embedded_103,synops

In [105]:
premiered_list = list(more_reduced_df['premiered'].astype(str))
p_list_no_zeros = [val for val in premiered_list if val != '0']
oldest = int((min(p_list_no_zeros)).split()[1])
print(oldest)
youngest = int((max(p_list_no_zeros)).split()[1])
print(youngest)

1968
2019


In [106]:
YR_RANGE = youngest - oldest + 1    # 2019 - 1968 + 1 = 52
SEASONS = ['Spring', 'Summer', 'Fall', 'Winter']
all_yrs = np.empty((0, YR_RANGE))   # shape = (0,52), ind0 = 1968
all_seasons = np.empty((0, len(SEASONS)))   # shape = (0, 4)
premiered = more_reduced_df['premiered'].astype(str)
for i in range(len(more_reduced_df.index)): # 0 to 2854
    premiered_year = np.zeros((YR_RANGE,))
    premiered_season = np.zeros((len(SEASONS),))
    splits = premiered.iloc[i].split()
    if(len(splits) != 1):
        premiered_year[int(splits[1])-oldest] = 1
        premiered_season[SEASONS.index(splits[0])] = 1
    all_yrs = np.append(all_yrs, premiered_year.reshape(1, -1),axis=0)
    all_seasons = np.append(all_seasons, premiered_season.reshape(1, -1),axis=0)
print(all_yrs.shape)
print(all_seasons.shape)

(2855, 52)
(2855, 4)


In [107]:
for i in range(all_yrs.shape[1]):
    more_reduced_df.insert(len(more_reduced_df.columns), "premiered_{}".format(i+oldest), all_yrs[:,i])
for i in range(all_seasons.shape[1]):
    more_reduced_df.insert(len(more_reduced_df.columns), "premiered_{}".format(SEASONS[i]), all_seasons[:,i])
print(more_reduced_df.shape)
more_reduced_df.head()

(2855, 465)


,animeID,episodes,premiered,score,scored_by,rank,popularity,members,favorites,genre_Action,genre_Adventure,genre_Comedy,genre_Dementia,genre_Drama,genre_Ecchi,genre_Fantasy,genre_Game,genre_Harem,genre_Historical,genre_Horror,genre_Kids,genre_Magic,genre_Military,genre_Music,genre_Mystery,genre_Psychological,genre_Romance,genre_Sci-Fi,genre_Slice of Life,genre_Sports,"studio_""Brains Base""",studio_A-1 Pictures,studio_AIC,studio_Arms,studio_Bones,studio_Diomedea,studio_Doga Kobo,studio_Gainax,studio_Gonzo,studio_J.C.Staff,studio_Kyoto Animation,studio_MAPPA,studio_Madhouse,studio_Manglobe,studio_Nippon Animation,studio_OLM,studio_P.A. Works,studio_Production I.G,studio_Production Reed,studio_Satelight,studio_Shaft,studio_Shin-Ei Animation,studio_Silver Link.,studio_Studio Deen,studio_Studio Gallop,studio_Studio Gokumi,studio_Studio Pierrot,studio_Sunrise,studio_TMS Entertainment,studio_TNK,studio_Tatsunoko Production,studio_Toei Animation,studio_Xebec,studio_Zexcs,studio_feel.,studio_ufotable,source_Game,source_Light novel,source_Manga,source_Novel,source_Original,source_Unknown,source_Visual novel,producer_Aniplex,producer_Bandai Visual,producer_DAX Production,producer_Dentsu,producer_Frontier Works,producer_Fuji TV,producer_Genco,producer_Geneon Universal Entertainment,producer_Kadokawa Shoten,producer_Lantis,producer_Media Factory,producer_Movic,producer_NHK,producer_Nihon Ad Systems,producer_Pony Canyon,producer_Production I.G,producer_Shochiku,producer_Shogakukan Productions,producer_Sotsu,producer_Starchild Records,producer_TBS,producer_TV Asahi,producer_TV Tokyo,producer_VAP,producer_Victor Entertainment,rating_G - All Ages,rating_PG - Children,rating_PG-13 - Teens 13 or older,rating_R - 17+ (violence & profanity),rating_R+ - Mild Nudity,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,synopsis_embedded_1,synopsis_embedded_2,synopsis_embedded_3,synopsis_embedded_4,synopsis_embedded_5,synopsis_embedded_6,synopsis_embedded_7,synopsis_embedded_8,synopsis_embedded_9,synopsis_embedded_10,synopsis_embedded_11,synopsis_embedded_12,synopsis_embedded_13,synopsis_embedded_14,synopsis_embedded_15,synopsis_embedded_16,synopsis_embedded_17,synopsis_embedded_18,synopsis_embedded_19,synopsis_embedded_20,synopsis_embedded_21,synopsis_embedded_22,synopsis_embedded_23,synopsis_embedded_24,synopsis_embedded_25,synopsis_embedded_26,synopsis_embedded_27,synopsis_embedded_28,synopsis_embedded_29,synopsis_embedded_30,synopsis_embedded_31,synopsis_embedded_32,synopsis_embedded_33,synopsis_embedded_34,synopsis_embedded_35,synopsis_embedded_36,synopsis_embedded_37,synopsis_embedded_38,synopsis_embedded_39,synopsis_embedded_40,synopsis_embedded_41,synopsis_embedded_42,synopsis_embedded_43,synopsis_embedded_44,synopsis_embedded_45,synopsis_embedded_46,synopsis_embedded_47,synopsis_embedded_48,synopsis_embedded_49,synopsis_embedded_50,synopsis_embedded_51,synopsis_embedded_52,synopsis_embedded_53,synopsis_embedded_54,synopsis_embedded_55,synopsis_embedded_56,synopsis_embedded_57,synopsis_embedded_58,synopsis_embedded_59,synopsis_embedded_60,synopsis_embedded_61,synopsis_embedded_62,synopsis_embedded_63,synopsis_embedded_64,synopsis_embedded_65,synopsis_embedded_66,synopsis_embedded_67,synopsis_embedded_68,synopsis_embedded_69,synopsis_embedded_70,synopsis_embedded_71,synopsis_embedded_72,synopsis_embedded_73,synopsis_embedded_74,synopsis_embedded_75,synopsis_embedded_76,synopsis_embedded_77,synopsis_embedded_78,synopsis_embedded_79,synopsis_embedded_80,synopsis_embedded_81,synopsis_embedded_82,synopsis_embedded_83,synopsis_embedded_84,synopsis_embedded_85,synopsis_embedded_86,synopsis_embedded_87,synopsis_embedded_88,synopsis_embedded_89,synopsis_embedded_90,synopsis_embedded_91,synopsis_embedded_92,synopsis_embedded_93,synopsis_embedded_94,synopsis_embedded_95,synopsis_embedded_96,synopsis_embedded_97,synopsis_embedded_98,synopsis_embedded_99,synopsis_embedded_100,synopsis_embedded_101,synopsis_embedded_102,synopsis_embedded_103,synops

In [108]:
# remove premiered column (no longer needed)
more_reduced_df = more_reduced_df.drop(['premiered'], axis = 1)
more_reduced_df.head()

# write back to csv
more_reduced_df.to_csv(r'data/model_training_data.csv')